In [1]:
%cd ..
import os
import time

/home/yongming/apperception


In [2]:
from apperception.database import database
from apperception.world import empty_world
from apperception.utils import F
from apperception.predicate import camera, objects
from optimized_ingestion.utils.preprocess import preprocess
database.connection
from optimized_ingestion.cache import disable_cache
disable_cache()

CUDA is available.
 > 0: Tesla T4


In [3]:
NUSCENES_PROCESSED_DATA = "NUSCENES_PROCESSED_DATA"
if NUSCENES_PROCESSED_DATA in os.environ:
    DATA_DIR = os.environ[NUSCENES_PROCESSED_DATA]
else:
    DATA_DIR = "/work/apperception/data/nuScenes/full-dataset-v1.0/Mini"
NUSCENES_RAW_DATA = "NUSCENES_RAW_DATA"
if NUSCENES_RAW_DATA in os.environ:
    RAW_DATA_DIR = os.environ[NUSCENES_RAW_DATA]
else:
    RAW_DATA_DIR = "/work/apperception/data/raw/nuScenes/full-dataset-v1.0/Mini"

In [4]:
import cv2
from os.path import exists
def construct_video(frames, size=(1600, 900), base=False, vid_name=None, vid_prefix=False):
    unique_frames = []
    for f in frames:
        if f not in unique_frames:
            unique_frames.append(f)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    vid_name = "./outputs/fig13_"+vid_name if vid_name else f"./outputs/{'fig13_base' if base else 'fig13_optimized'}.mp4"
    display_video = cv2.VideoWriter(vid_name,fourcc, 1, size)
    for frame in unique_frames:
        if vid_prefix:
            img_path = os.path.join(RAW_DATA_DIR, 'sweeps/CAM_FRONT', frame)
            if not exists(img_path):
                img_path = os.path.join(RAW_DATA_DIR, 'samples/CAM_FRONT', frame)
        else:
            img_path = os.path.join(RAW_DATA_DIR, frame)
        img = cv2.imread(img_path)
        display_video.write(img)

    display_video.release()

In [5]:
name = 'ScenicWorld'
world = empty_world(name=name)

obj1 = objects[0]
obj2 = objects[1]
cam = camera

# world = world.filter(
#     (obj1.id != obj2.id) &
#     (F.like(obj1.type, 'car') | F.like(obj1.type, 'truck')) &
#     (F.like(obj2.type, 'car') | F.like(obj2.type, 'truck')) &
#     F.angle_between(F.facing_relative(cam.ego, F.road_direction(cam.ego)), -15, 15) &
#     (F.distance(cam.ego, obj1.trans@cam.time) < 50) &
#     (F.view_angle(obj1.trans@cam.time, cam.ego) < 70 / 2.0) &
#     (F.distance(cam.ego, obj2.trans@cam.time) < 50) &
#     (F.view_angle(obj2.trans@cam.time, cam.ego) < 70 / 2.0) &
#     F.contains_all('intersection', [obj1.trans, obj2.trans]@cam.time) &
#     F.angle_between(F.facing_relative(obj1.trans@cam.time, cam.ego), 40, 135) &
#     F.angle_between(F.facing_relative(obj2.trans@cam.time, cam.ego), -135, -50) &
#     (F.min_distance(cam.ego, 'intersection') < 10) &
#     F.angle_between(F.facing_relative(obj1.trans@cam.time, obj2.trans@cam.time), 100, -100)
# )
world = world.filter(
    (F.like(obj1.type, 'car') | F.like(obj1.type, 'truck')) &
    F.angle_between(F.facing_relative(cam.ego, F.road_direction(cam.ego)), -15, 15) &
    (F.distance(cam.ego, obj1.trans@cam.time) < 50) &
    F.contains_all('intersection', [obj1.trans]@cam.time) &
    F.angle_between(F.facing_relative(obj1.trans@cam.time, cam.ego), 135, 225) &
    (F.min_distance(cam.ego, 'intersection') < 10)
)


In [6]:
preprocess(world, DATA_DIR, ['scene-0655-CAM_FRONT'], base=False)

scene-0655-CAM_FRONT --------------------------------------------------------------------------------
Stage:  InView
Stage:  DecodeFrame.ParallelDecodeFrame


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.21s/it]


Stage:  Detection2D.YoloDetection


ego_speed: 8.223887668036209

    WITH
    Point AS (SELECT * FROM UNNEST(ARRAY[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15],ARRAY[1776.7123969743516,1777.3754208279192,1777.5167320592332,1777.2650195969686,1777.5150646081534,1777.530511129799,1777.22766947455,1777.3814765030393,1777.6314865781696,1777.6917515267833,1777.455664469456,1777.6403527094342,1777.7010468480094,1776.9993140584243,1777.6349137894322,1777.6534724892797],ARRAY[868.16619194755,867.7135512712596,867.1361675203339,864.686272618922,865.8185386645551,865.3253686022381,865.1370397982982,867.2621373685764,866.9997781466614,866.616111195383,867.1897224556392,866.8831067924314,866.4943535773085,864.6515900223155,866.8536456427435,866.710460749178],ARRAY[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]) AS _point (token, tx, ty, tz)),
    AvailablePolygon AS (
        SELECT *
        FROM SegmentPolygon
        WHERE location = 'boston-seaport'
        AND SegmentPolygon.__RoadType__intersection__
        AND S

Stage:  Detection2D.ObjectTypeFilter
Stage:  Detection3D.FromDetection2DAndRoad
Stage:  DetectionEstimation


ValueError: 

In [6]:
optimized_query_start = time.time()
id_time_camId_filename = world.get_id_time_camId_filename(1)
print(f'optimized query time: {time.time()-optimized_query_start}')

execute: filter
execute: get_id_time_camId_filename
get_id_time_camId_filename 
        SELECT t0.itemId, cameras.timestamp, cameras.cameraId, cameras.filename
        FROM (SELECT * FROM item_general_trajectory) as t0
        
        JOIN Cameras USING (cameraId)
        WHERE ((t0.objectType LIKE 'car' OR t0.objectType LIKE 'truck') AND angleBetween(facingRelative(egoHeading,roadDirection(egoTranslation,egoHeading)),-15,15) AND (ST_Distance(egoTranslation,valueAtTimestamp(t0.translations,timestamp))<50) AND (EXISTS(SELECT 1
            FROM SegmentPolygon
            WHERE
                SegmentPolygon.__RoadType__intersection__ AND
                ST_Covers(SegmentPolygon.elementPolygon, valueAtTimestamp(t0.translations,timestamp))
        )) AND angleBetween(facingRelative((headingAtTimestamp(t0.itemHeadings,timestamp))::real,egoHeading),135,225) AND (minDistance(egoTranslation,'intersection')<10))
        
done execute node
Result length: 10
optimized query time: 72.545421600341

In [7]:
optimized_results = [e[3] for e in id_time_camId_filename]
construct_video(optimized_results, vid_name="modified.mp4")

In [ ]:
preprocess(world, DATA_DIR, ['scene-0757-CAM_FRONT'])

In [ ]:
base_query_start = time.time()
id_time_camId_filename_base = world.get_id_time_camId_filename(2)
print(f'based query time: {time.time()-base_query_start}')

In [ ]:
id_time_camId_filename_base

In [ ]:
base_results = [e[4] for e in id_time_camId_filename_base]
construct_video(base_results, base=True)

In [ ]:
false_negative = [f for f in base_results if f not in optimized_results]
print(len(false_negative))
construct_video(false_negative, vid_name="false_negative.avi")

In [ ]:
false_positive = [f for f in optimized_results if f not in base_results]
print(len(false_positive))
construct_video(false_positive, vid_name="false_positive.avi")